In [532]:
# from thermo.chemical import Chemical

# emissivity = 0
# temperature = 1900 #kelvin

# boltzmann_constant = 5.6703e-10 

# test this is 

# aluminium = Chemical('aluminium')

# aluminium_vapor_pressure = aluminium.VaporPressure.calculate(2000,"BOILING_CRITICAL") # takes temp in K, returns pressure in Pa
#                                                                                       # not accurate!
# print(aluminium_vapor_pressure)

# graphite = Chemical('carbon|solid')
# graphite_vapor_pressure = graphite.VaporPressure.T_dependent_property(1900) # takes temp in K, returns pressure in Pa
# print(graphite_vapor_pressure)

In [533]:
from mendeleev import element
from thermo.chemical import Chemical
import sympy.physics.units as units
import scipy.constants as constants
amu = constants.physical_constants["atomic mass unit-kilogram relationship"][0]
import math
import sys
sys.path.insert(0, '../../files/ionprinter/simulation/utilities/')
import util

In [534]:
#Bowtie inputs:

print_species = element('Aluminum')

print_speed = 10 #g/hour

print_speed /= 1000.0
print_speed /= 3600.0

number_of_bowties = 10000.0

bowtie_nozzle_radius = 0.00033#m

bowtie_hot_area = 40e-6 #m^2
graphite_emissivity = 0.95

bowtie_temperature = 1600.0 #Kelvin

# import pandas as pd
# data = [[1, 2], [3, 4]]
# pd.DataFrame(data, columns=["Foo", "Bar"])


#chamber inputs

ionization_chamber_radius = 0.0015 #m
ionization_chamber_length = 0.01 #m
acceleration_voltage = 100 #v
ionization_efficiency = 0.3

ICP_frequency = 1000000
ICP_peak_B = 0.05

build_platform_distance = 0.03 #m

In [535]:
atoms_per_second = print_speed/(print_species.mass*amu)
moles_per_second = atoms_per_second/constants.Avogadro
print_volume = ((print_speed*3600.0)/(print_species.density*1000.0))/1.0e-6

total_beam_current = atoms_per_second*constants.elementary_charge
per_chamber_beam_current = total_beam_current/number_of_bowties

print("Atoms per second: \nper chamber %0.5g atoms/s \ntotal %0.5g atoms/s" % ((atoms_per_second/number_of_bowties),
                                           atoms_per_second))
print("Total moles per second: %0.5g M/s" % moles_per_second)
print("Print volume: %0.5g cm^3/h" % print_volume)
print("Total beam current: %0.5g Amps" % total_beam_current)
print("Per-chamber current: %0.5g Amps" % per_chamber_beam_current)

#I find it infinitely amusing that "40 amps of aluminum" is "40 grams per second of aluminum" within 0.6%

Atoms per second: 
per chamber 6.1999e+15 atoms/s 
total 6.1999e+19 atoms/s
Total moles per second: 0.00010295 M/s
Print volume: 3.7052 cm^3/h
Total beam current: 9.9333 Amps
Per-chamber current: 0.00099333 Amps


Determine RMS neutral gas velocity at a certain temperature:

$$\sqrt{\frac{3RT}{m}}$$

In [536]:
bowtie_rms_neutral_velocity = math.sqrt((3*constants.Boltzmann*bowtie_temperature)/(print_species.mass*amu))
bowtie_rms_energy = 0.5*print_species.mass*amu*(bowtie_rms_neutral_velocity**2.0)
print("RMS particle velocity: %0.5g m/s" % bowtie_rms_neutral_velocity)
print("RMS particle energy: %0.5g eV" % (bowtie_rms_energy/constants.electron_volt))


RMS particle velocity: 1216.2 m/s
RMS particle energy: 0.20682 eV


Determine approximate neutral gas pressure:

(I'm not even sure that you can do this.)
$$\text{PV}=\text{nRT}$$

In [537]:
R_constant = constants.Boltzmann*constants.Avogadro

nozzle_volume = (math.pi*(bowtie_nozzle_radius**2))*bowtie_rms_neutral_velocity

bowtie_nozzle_pressure = ((moles_per_second/number_of_bowties)*R_constant*bowtie_temperature)/(nozzle_volume)

mean_free_path = 1.0/(((atoms_per_second/number_of_bowties)/nozzle_volume)*math.pi*((print_species.vdw_radius*1e-12)**2))

vapor_pressure = Chemical(print_species.name).VaporPressure.calculate(bowtie_temperature,"BOILING_CRITICAL") # takes temp in K, returns pressure in P

print("Bowtie nozzle pressure: %0.5g Pa" % bowtie_nozzle_pressure)
print("Approx. species vapor pressure: %0.5g Pa" % vapor_pressure)
print("Approximate nozzle mean free path: %0.5g m" % mean_free_path)
print("nrho: %0.5g" % (atoms_per_second/nozzle_volume))


print("Knudsen number: local %0.5g, global %0.5g" % ((mean_free_path/bowtie_nozzle_radius),
                                                     (mean_free_path/build_platform_distance)))

vapor_pressure = Chemical('aluminium').VaporPressure.calculate(bowtie_temperature,"BOILING_CRITICAL") # takes temp in K, returns pressure in Pa
                                                                                       # not accurate!
print("Worst-case space charge density: %0.5g" % ((atoms_per_second/number_of_bowties)/nozzle_volume))

Bowtie nozzle pressure: 0.32916 Pa
Approx. species vapor pressure: 10.296 Pa
Approximate nozzle mean free path: 0.63098 m
nrho: 1.49e+23
Knudsen number: local 1912.1, global 21.033
Worst-case space charge density: 1.49e+19


In [538]:
acceleration_velocity = math.sqrt((2.0*acceleration_voltage*constants.electron_volt)/(print_species.mass*amu))
print("Accel. vel: %0.5g m/s" % acceleration_velocity)
print("Accel. power per chamber: %0.5g W, total %0.5g W " % (acceleration_voltage*per_chamber_beam_current
                                                        ,acceleration_voltage*total_beam_current))

beam_exit_velocity = bowtie_rms_neutral_velocity+acceleration_velocity

focus_field = util.scharge_efield(per_chamber_beam_current,beam_exit_velocity,ionization_chamber_radius)
print("Required focus field: %0.5g V/m" % focus_field)

#sine focus voltage equation would be nice
print("Required focus voltage: ~%0.5g V" % (focus_field*(ionization_chamber_radius/5.0)))


Accel. vel: 26743 m/s
Accel. power per chamber: 0.099333 W, total 993.33 W 
Required focus field: 4.2594e+05 V/m
Required focus voltage: ~127.78 V


In [539]:
per_chamber_ionization_power = ((atoms_per_second/number_of_bowties)*print_species.ionenergies[1]*constants.electron_volt)
total_ionization_power = per_chamber_ionization_power*number_of_bowties
print("Ionization power: per ~%0.5g W, total %0.5g W" % (per_chamber_ionization_power,total_ionization_power))

Ionization power: per ~0.0059458 W, total 59.458 W


Bowtie radiation power:

In [540]:
per_bowtie_power = (bowtie_hot_area * graphite_emissivity * constants.Stefan_Boltzmann * (bowtie_temperature**4.0))
total_bowtie_power = per_bowtie_power*number_of_bowties

print("Bowtie power: per %0.5g W, total %0.5g W" % (per_bowtie_power,total_bowtie_power))


Bowtie power: per 14.121 W, total 1.4121e+05 W


In [541]:
total_power_consumption = total_bowtie_power + (acceleration_voltage*total_beam_current) + per_chamber_ionization_power
print("Total power consumption: %0.5g W" % (total_power_consumption))

Total power consumption: 1.4221e+05 W


ICP acceleration field, (line integral of Maxwell's second equation, eq. 4.5-3 of "Fundamentals"):

In [542]:
(-(1.0j*2.0*math.pi*ICP_ionization_frequency)/2.0)*

SyntaxError: invalid syntax (<ipython-input-542-2bfe023269aa>, line 1)